Importing basic libraries for data processing

In [1]:
import numpy as np
import pandas as pd

Loading the dataset

In [2]:
data = pd.read_csv('./Dataset/Existing Base.csv')
data.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


In [3]:
data.shape

(10155, 32)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10155 entries, 0 to 10154
Data columns (total 32 columns):
REF_NO                             10155 non-null int64
children                           10155 non-null object
age_band                           10155 non-null object
status                             10155 non-null object
occupation                         10155 non-null object
occupation_partner                 10155 non-null object
home_status                        10155 non-null object
family_income                      10155 non-null object
self_employed                      10155 non-null object
self_employed_partner              10155 non-null object
year_last_moved                    10155 non-null int64
TVarea                             10155 non-null object
post_code                          10155 non-null object
post_area                          10155 non-null object
Average Credit Card Transaction    10155 non-null float64
Balance Transfer                   10

In [5]:
len(data.select_dtypes(['object']).columns)

14

We are having 32 columns out of them 14 are categorical. Lets look each variable one by one

First we are having Ref.No we can drop that variable so keep that aside as of now and lets look into 2nd feature which is *children*.

In [6]:
data['children'].value_counts()

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

In [7]:
data.loc[data['children'] == 'Zero','children'] = '0'
data.loc[data['children'] == '4+','children'] = '4'
data['children'] = pd.to_numeric(data['children'])
data['children'].dtype

dtype('int64')

In [8]:
data['children'].value_counts()

0    6208
1    1848
2    1607
3     473
4      19
Name: children, dtype: int64

Our Target variable is Revenue.Grid so converting value 2 to 0 and 1 to 1

In [9]:
data['Revenue Grid'].value_counts()

2    9069
1    1086
Name: Revenue Grid, dtype: int64

In [10]:
data['Revenue Grid'] = np.where(data['Revenue Grid'] == 2,0,1)
data['Revenue Grid'].value_counts

<bound method IndexOpsMixin.value_counts of 0        1
1        0
2        0
3        0
4        0
        ..
10150    0
10151    0
10152    0
10153    0
10154    0
Name: Revenue Grid, Length: 10155, dtype: int64>

Now lets move to age_band variable

In [11]:
data['age_band'].value_counts()

45-50      1359
36-40      1134
41-45      1112
31-35      1061
51-55      1052
55-60      1047
26-30       927
61-65       881
65-70       598
22-25       456
71+         410
18-21        63
Unknown      55
Name: age_band, dtype: int64

considering the age as a categorical variable, so instead of taking mean value just creating dummies based on the response on target variable

In [12]:
round(data.groupby('age_band')['Revenue Grid'].mean(),2)

age_band
18-21      0.17
22-25      0.11
26-30      0.11
31-35      0.11
36-40      0.13
41-45      0.11
45-50      0.10
51-55      0.10
55-60      0.11
61-65      0.09
65-70      0.10
71+        0.10
Unknown    0.05
Name: Revenue Grid, dtype: float64

In [13]:
for i in range(len(data)):
    if data["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        data.loc[i,"age_band"]="ab_10"
    if data["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        data.loc[i,"age_band"]="ab_11"
    if data["age_band"][i]=="36-40":
        data.loc[i,"age_band"]="ab_13"
    if data["age_band"][i]=="18-21":
        data.loc[i,"age_band"]="ab_17"
    if data["age_band"][i]=="61-65":
        data.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(data["age_band"])
ab_dummies.head()

,Unknown,ab_10,ab_11,ab_13,ab_17,ab_9
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0


In [14]:
ab_dummies.drop('Unknown',axis='columns',inplace=True)

In [15]:
data = pd.concat((data,ab_dummies),axis='columns')

In [16]:
data.drop('age_band',axis='columns',inplace=True)

We have just created dummies age_band by grouping them. Now go for status feature

In [17]:
data['status'].value_counts()

Partner                 7709
Single/Never Married    1101
Divorced/Separated       679
Widowed                  618
Unknown                   48
Name: status, dtype: int64

For this status feature also lets create dummies by not creating dummies for widowed and unknown

In [18]:
data['status_partner'] = np.where(data['status'] == 'Partner',1,0)
data['statuss_S/NM'] = np.where(data['status'] == 'Single/Never Married',1,0)
data['status_div'] = np.where(data['status'] == 'Divorced/Separated',1,0)
data.drop('status',axis='columns')

,REF_NO,children,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,year_last_moved,TVarea,...,Investment in Derivative,Portfolio Balance,ab_10,ab_11,ab_13,ab_17,ab_9,status_partner,statuss_S/NM,status_div
0,1,0,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,1972,HTV,...,32.32,89.43,1,0,0,0,0,1,0,0
1,2,0,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,1998,Granada,...,4.33,22.78,0,1,0,0,0,0,1,0
2,3,0,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,1996,Tyne Tees,...,80.96,171.78,0,1,0,0,0,0,1,0
3,5,0,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,1997,Central,...,17.57,-41.70,0,0,0,1,0,0,1,0
4,6,0,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,1995,Scottish TV,...,80.44,235.02,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10150,11512,1,Secretarial/Admin,Manual Worker,Own Home,"<30,000, >=27,500",No,No,1972,Carlton,...,9.57,10.23,1,0,0,0,0,1,0,0
10151,11513,0,Manual Worker,Manual Worker,Rent from Council/HA,"<25,000, >=22,500",No,No,1988,Meridian,...,36.40,102.62,1,0,0,0,0,1,0,0
10152,11514,2,Housewife,Professional,Own Home,">=35,000",No,No,1992,Central,...,14.07,76.18,0,1,0,0,0,1,0,0
10153,11516,0,Other,Manual Worker,Own Home,"<10,000, >= 8,000",No,Yes,1970,Carlton,...,1.66,4.79,1,0,0,0,0,1,0,0


For occupation and occupation partner we also create dummies by group with the response rate

In [19]:
data['occupation'].value_counts()

Professional         2449
Retired              2206
Secretarial/Admin    1797
Housewife            1255
Business Manager      732
Unknown               567
Manual Worker         556
Other                 537
Student                56
Name: occupation, dtype: int64

In [20]:
round(data.groupby('occupation')['Revenue Grid'].mean(),2)

occupation
Business Manager     0.12
Housewife            0.09
Manual Worker        0.11
Other                0.11
Professional         0.12
Retired              0.10
Secretarial/Admin    0.11
Student              0.11
Unknown              0.11
Name: Revenue Grid, dtype: float64

In [21]:
for i in range(len(data)):
    if data['occupation'][i] in ['Business Manager','Professional']:
        data.loc[i,'occupation'] = 'occ_12'
    if data['occupation'][i] in ['Manual Worker','Other','Secretarial/Admin','Student','Unknown']:
        data.loc[i,'occupation'] = 'occ_11'
    if data['occupation'][i] in ['Retired']:
        data.loc[i,'occupation'] = 'occ_9'

d = pd.get_dummies(data['occupation'])
d

,Housewife,occ_11,occ_12,occ_9
0,0,1,0,0
1,0,0,0,1
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
10150,0,1,0,0
10151,0,1,0,0
10152,1,0,0,0
10153,0,1,0,0


In [22]:
data = pd.concat([data,d],axis='columns')

In [23]:
data.drop(['Housewife','REF_NO','status'],axis='columns',inplace=True)

In [24]:
data.drop('occupation',axis='columns',inplace=True)

In [25]:
data['occupation_partner'].value_counts()

Unknown              2394
Professional         2051
Retired              1936
Manual Worker        1508
Business Manager      737
Secretarial/Admin     662
Housewife             527
Other                 324
Student                16
Name: occupation_partner, dtype: int64

In [26]:
round(data.groupby('occupation_partner')['Revenue Grid'].mean(),2)

occupation_partner
Business Manager     0.11
Housewife            0.11
Manual Worker        0.11
Other                0.10
Professional         0.11
Retired              0.10
Secretarial/Admin    0.12
Student              0.12
Unknown              0.10
Name: Revenue Grid, dtype: float64

In [27]:
for i in range(len(data)):
    if data['occupation_partner'][i] in ['Business Manager','Housewife','Manual Worker','Professional']:
        data.loc[i,'occupation_partner'] = 'ooc_ptr_11'
    if data.loc[i,'occupation_partner'] in ['Secretarial/Admin','Student']:
        data.loc[i,'occupation_partner'] = 'ooc_ptr_12'
d = pd.get_dummies(data['occupation_partner'])
d

,Other,Retired,Unknown,ooc_ptr_11,ooc_ptr_12
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,0,0,1,0
4,0,0,1,0,0
...,...,...,...,...,...
10150,0,0,0,1,0
10151,0,0,0,1,0
10152,0,0,0,1,0
10153,0,0,0,1,0


In [28]:
data = pd.concat((data,d),axis=1)

In [29]:
data.drop(['Other','Retired','Unknown','occupation_partner'],axis=1,inplace=True)

Converting self_employed,self_employed_partner  to numeric

In [30]:
data['self_employed'] = np.where(data['self_employed'] == 'Yes',1,0)
data['self_employed_partner'] = np.where(data['self_employed_partner'] == 'Yes',1,0)

Now lets look into home_status

In [31]:
data['home_status'].value_counts()

Own Home                9413
Rent from Council/HA     322
Rent Privately           261
Live in Parental Hom     109
Unclassified              50
Name: home_status, dtype: int64

Own home are widly spread so we can drop the rest of the categoru and make own home as 1

In [32]:
data['home_status'] = np.where(data['home_status'] =='Own Home',1,0)

In [33]:
data['TVarea'].value_counts()

Central          1618
Carlton          1541
Meridian         1226
Yorkshire        1042
Granada          1039
HTV               845
Anglia            751
Tyne Tees         536
Scottish TV       520
TV South West     363
Grampian          227
Unknown           189
Ulster            159
Border             99
Name: TVarea, dtype: int64

In [34]:
data['post_area'].nunique()

2039

In [35]:
data['post_code'].nunique()

10040

In [36]:
data['region'].value_counts()

South East          2100
North West          1896
Unknown             1080
South West           971
West Midlands        820
Scotland             787
East Midlands        771
North                571
Wales                556
East Anglia          423
Northern Ireland     159
Isle of Man           16
Channel Islands        5
Name: region, dtype: int64

dropping TVarea, postcode,postarea,region. if model is not well performing we'll generate dummies for TVarea and region

In [37]:
data.drop(['post_area','post_code','region','TVarea'],axis='columns',inplace=True)

Now lets see gender variable

In [38]:
data['gender'].value_counts()

Female     7634
Male       2486
Unknown      35
Name: gender, dtype: int64

In [39]:
data['gender'] = np.where(data['gender'] == 'Female',0,1)

Now we left out with family income feature.

In [40]:
data['family_income'].value_counts()

>=35,000             2517
<27,500, >=25,000    1227
<30,000, >=27,500     994
<25,000, >=22,500     833
<20,000, >=17,500     683
<12,500, >=10,000     677
<17,500, >=15,000     634
<15,000, >=12,500     629
<22,500, >=20,000     590
<10,000, >= 8,000     563
< 8,000, >= 4,000     402
< 4,000               278
Unknown               128
Name: family_income, dtype: int64

In [41]:
data["fi"]=4 # by doing this , we have essentially clubbed <4000 and Unknown values . How?
data.loc[data["family_income"]=="< 8,000, >= 4,000","fi"]=6
data.loc[data["family_income"]=="<10,000, >= 8,000","fi"]=9
data.loc[data["family_income"]=="<12,500, >=10,000","fi"]=11.25
data.loc[data["family_income"]=="<15,000, >=12,500","fi"]=13.75
data.loc[data["family_income"]=="<17,500, >=15,000","fi"]=16.25
data.loc[data["family_income"]=="<20,000, >=17,500","fi"]=18.75
data.loc[data["family_income"]=="<22,500, >=20,000","fi"]=21.25
data.loc[data["family_income"]=="<25,000, >=22,500","fi"]=23.75
data.loc[data["family_income"]=="<27,500, >=25,000","fi"]=26.25
data.loc[data["family_income"]=="<30,000, >=27,500","fi"]=28.75
data.loc[data["family_income"]==">=35,000","fi"]=35
data=data.drop(["family_income"],axis=1)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10155 entries, 0 to 10154
Data columns (total 36 columns):
children                           10155 non-null int64
home_status                        10155 non-null int64
self_employed                      10155 non-null int64
self_employed_partner              10155 non-null int64
year_last_moved                    10155 non-null int64
Average Credit Card Transaction    10155 non-null float64
Balance Transfer                   10155 non-null float64
Term Deposit                       10155 non-null float64
Life Insurance                     10155 non-null float64
Medical Insurance                  10155 non-null float64
Average A/C Balance                10155 non-null float64
Personal Loan                      10155 non-null float64
Investment in Mutual Fund          10155 non-null float64
Investment Tax Saving Bond         10155 non-null float64
Home Loan                          10155 non-null float64
Online Purchase Amount         

In [43]:
data.dropna(inplace=True)

## Splitting the data into train and test

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
train,test = train_test_split(data,test_size=0.2,random_state=4)

In [46]:
x_train = train.drop('Revenue Grid',axis=1)
y_train = train['Revenue Grid']

x_test = test.drop('Revenue Grid',axis=1)
y_test = test['Revenue Grid']

## Decision Trees

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [51]:
dtree = DecisionTreeClassifier(criterion = 'entropy',max_depth=7,random_state=6,class_weight='balanced')

In [52]:
dtree.fit(x_train,y_train,)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=6, splitter='best')

In [54]:
pred_inCLass = dtree.predict(x_test)
pred_proba = dtree.predict_proba(x_test)

Now we have predictions in our hand lets check for the model performance

### Performance metrics for Classification model

In [79]:
class Classification_Performance:
    '''Logistic_Performance(actual=<true values>,predicted=<model outcomes>)
        method -> accuracy() 
                    return accuracy of logistic model
        method -> confusion_matrix()
                    return TN,FP,FN,TP
        method -> precision_recall
                    return precision and recall of the model
        method -> ks_score()
        
        method -> f1_score()
        
        method -> fBeta_score()
        
        method -> roc_auc()
                    '''
    def __init__(self,actual,predicted):
        self.actual = actual
        self.predicted = predicted
    
    def accuracy(self):
        self.acc = metrics.accuracy_score(self.actual,self.predicted)
        print('Accuracy : ',self.acc)
        
    def confusion_matrix(self):
        cm = metrics.confusion_matrix(self.actual,self.predicted,)
        self.TN,self.FP,self.FN,self.TP = cm.ravel()
        print('True Negative : ',self.TN)
        print('False Positive : ',self.FP)
        print('False Negative : ',self.FN)
        print('True Positive : ',self.TP)
        
    def precision_recall(self):
        self.precision = self.TP/(self.TP + self.FP)
        self.recall = self.TP/(self.TP + self.FN)
        
        print("Precision : ",self.precision)
        print("Recall : ",self.recall)
        
        
    def ks_score(self):
        self.KS_score = self.recall - self.FP/(self.FP + self.TN)
        print("KS_score : ",self.KS_score)
        
    def f1_score(self):
        self.f1 = metrics.f1_score(self.actual,self.predicted)
        print("F1 Score : ",self.f1)
        
    def fBeta_score(self,Beta):
        """
        The `beta` parameter determines the weight of recall in the combined
score. ``beta < 1`` lends more weight to precision, while ``beta > 1``
favors recall (``beta -> 0`` considers only precision, ``beta -> inf``
only recall).
        """
        self.fBeta = metrics.fbeta_score(self.actual,self.predicted,beta=Beta)
        print("F Beta Score when Beta = ",Beta,"is = ",self.fBeta)
        
    def roc_auc(self):
        self.auc_roc = metrics.roc_auc_score(self.actual,self.predicted)
        print("Area under the curve = ",self.auc_roc)
        
    def log_loss(self,predicted_proba):
        self.logLoss = metrics.log_loss(self.actual,predicted_proba)
        print("My model is lacking the confidence of ",round(self.logLoss*100,2), "%")
        
    def check_all(self):
        self.accuracy()
        self.confusion_matrix()
        self.precision_recall()
        self.ks_score()
        self.f1_score()
        self.roc_auc()

In [58]:
a1 = Classification_Performance(y_test,pred_inCLass)

In [59]:
a1.accuracy()

Accuracy :  0.9290989660265879


In [60]:
a1.confusion_matrix()

True Negative :  1696
False Positive :  116
False Negative :  28
True Positive :  191


In [61]:
a1.precision_recall()

Precision :  0.6221498371335505
Recall :  0.8721461187214612


In [62]:
a1.f1_score()

F1 Score :  0.726235741444867


In [63]:
a1.roc_auc()

Area under the curve =  0.9040642293386555


In [65]:
a1.log_loss(pred_proba)

My model is lacking the confidence of  32.55 %


*Area under the curve is just cover 90% still we can reduce it with more advance algorithms*
In logistic Regression we got the below performance metric

**Logistic Regression:**         

Accuracy :  0.914327917282127

True Negative :  1663
False Positive :  149
False Negative :  25
True Positive :  194

Precision :  0.565597667638484
Recall :  0.8858447488584474

F1 Score :  0.6903914590747331

Area under the curve =  0.9018075841422479

My model is lacking the confidence of  25.86 %

**Decision Trees** (without CV and max_Depth 7)

Accuracy :  0.9290989660265879 (*Improved than Logistic Rergression*)

True Negative :  1696 (*Improved than Logistic*)

False Positive :  116 (*Improved than Logistic*)

False Negative :  28  (***Not Improved than Logistic***)

True Positive :  191  (***Not Improved than Logistic***)

Precision :  0.6221498371335505 (*Improved than Logistic*)

Recall :  0.8721461187214612 (***Not Improved than Logistic)


F1 Score :  0.726235741444867 (*Improved*)

Area under the curve =  0.9040642293386555 (***Not Improved***)

My model is lacking the confidence of  32.55 % (***Not predicting with high confidence***)

**Over all conclusion:**

As the data is imbalance so decision tree model slightly biased to class 0 due to high in number.

As we have only used max depth as hyper parameter, Now we will experiment it with various other hyperparameters and train the model with and without cross validation

**Tuning Hyper parameter max_depth without crossvalidation**

In [81]:
depth = [2,4,6,7,8,9,10,11,12,13,14,None]
score_f1 = []
for i in depth:
    tree = DecisionTreeClassifier(criterion='entropy',random_state=6,max_depth=i,class_weight='balanced')
    
    tree.fit(x_train,y_train)
    pred = tree.predict(x_test)
    performance = Classification_Performance(y_test,pred)
    performance.f1_score()
    score_f1.append(performance.f1)

my_depth = depth[score_f1.index(max(score_f1))]
print("Maximum F1 score achieved at depth : ",my_depth)

tree_final = DecisionTreeClassifier(criterion='entropy',random_state=6,max_depth=my_depth,
                                   class_weight='balanced')

tree_final.fit(x_train,y_train)
pred = tree_final.predict(x_test)
performance = Classification_Performance(y_test,pred)
performance.check_all()
performance.log_loss(tree_final.predict_proba(x_test))

F1 Score :  0.3886162904808636
F1 Score :  0.6354515050167224
F1 Score :  0.6721311475409837
F1 Score :  0.726235741444867
F1 Score :  0.758220502901354
F1 Score :  0.7351778656126483
F1 Score :  0.7478991596638654
F1 Score :  0.7534246575342466
F1 Score :  0.7482993197278911
F1 Score :  0.7378190255220418
F1 Score :  0.727699530516432
F1 Score :  0.748792270531401
Maximum F1 score achieved at depth :  8
Accuracy :  0.9384539635647464
True Negative :  1710
False Positive :  102
False Negative :  23
True Positive :  196
Precision :  0.6577181208053692
Recall :  0.8949771689497716
KS_score :  0.8386857782212949
F1 Score :  0.758220502901354
Area under the curve =  0.9193428891106474
My model is lacking the confidence of  48.54 %


**Tuning Hyper parameter with Cross validation**

In [83]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

In [136]:
params = {"max_depth" : [2,4,6,7,8,9,10,11,12,13,14]}

tree3 = DecisionTreeClassifier(criterion='entropy',random_state=6,max_depth=i,class_weight='balanced')

dtrees_rs = RandomizedSearchCV(tree3,param_distributions=params,
                      n_iter=len(params['max_depth']),
                  scoring='roc_auc',cv=3)

In [137]:
dtrees_rs.fit(x_train,y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight='balanced',
                                                    criterion='entropy',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=6,
                                                    splitter='

In [138]:
dtrees_rs.best_params_

{'max_depth': 6}

In [139]:
dtrees_rs.best_index_

2

In [140]:
dtrees_rs.best_score_

0.9397598691252879

In [142]:
sample = dtrees_rs.best_estimator_
sample.fit(x_train,y_train)
pred_inClass = sample.predict(x_test)
pred_proba = sample.predict_proba(x_test)
performance = Classification_Performance(y_test,pred_inClass)
performance.check_all()
performance.log_loss(pred_proba)

Accuracy :  0.9015263417035942
True Negative :  1626
False Positive :  186
False Negative :  14
True Positive :  205
Precision :  0.5242966751918159
Recall :  0.9360730593607306
KS_score :  0.8334240527382141
F1 Score :  0.6721311475409837
Area under the curve =  0.9167120263691071
My model is lacking the confidence of  29.22 %


## Over all conclusion

**Logistic Regression**

Accuracy : 0.914327917282127

Area under the curve = 0.9018075841422479

My model is lacking the confidence of 25.86 %

**Decision Tree with out CV and Max_depth = 7**

Accuracy : 0.9290989660265879

Area under the curve = 0.9040642293386555

My model is lacking the confidence of 32.55 % 

**Decision Tree with Hyper parameter-Max_Depth tuning and withourt CV

Accuracy :  0.9384539635647464

Area under the curve =  0.9193428891106474

My model is lacking the confidence of  48.54 %

**Decision Tree with CV and Hyper parameter-Max_Depth Tuning**

Accuracy :  0.9015263417035942

***Area under the curve =  0.9167120263691071***

*My model is lacking the confidence of  29.22 %*